In [127]:
from pathlib import Path

In [140]:
for dir in Path("results").glob("generator*"):
    fp = dir / "results.jsonl"
    if not fp.exists():
        continue
    data = pd.read_json(fp, lines=True)

    # fix stuff
    idx = data.prompt_fun == "prompt_human_vs_ai"
    data.loc[idx, "correct"] = (
        ((data[idx]["prediction"] == "fable1") & (data[idx]["generator1"] != "human")) | 
        ((data[idx]["prediction"] == "fable2") & (data[idx]["generator2"] != "human"))
    ).astype(float)

    # add correct prediction flag
    data.loc[data.correct==1.,"correct_prediction"] = data["prediction"].str.replace("fable1", "1").replace("fable2", "2").replace("1", "fable1").replace("2", "fable2")
    data.loc[data.correct==0.,"correct_prediction"] = data["prediction"].str.replace("fable1", "2").replace("fable2", "1").replace("1", "fable1").replace("2", "fable2")

    # export back to file
    fp_new = dir / "results_corrected.jsonl"
    data.to_json(fp_new, orient="records", lines=True)

In [145]:
!cat results/**/results_corrected.jsonl > results_combined.jsonl
!wc -l results_combined.jsonl

4771 results_combined.jsonl


In [146]:
import pandas as pd
data = pd.read_json("results_combined.jsonl", lines=True)

In [147]:
data.groupby("predictor").count()["id"]

predictor
claude-2         1890
gpt-3.5-turbo    1890
gpt-4             991
Name: id, dtype: int64

In [149]:
data.dropna().groupby(["predictor", "prediction", "correct_prediction", ])["prediction"].agg(["count"])

count
predictor     prediction correct_prediction       
claude-2      fable1     fable1                104
                         fable2                102
              fable2     fable1                684
                         fable2                721
              unknown    unknown                12
gpt-3.5-turbo fable1     fable1                835
                         fable2                555
              fable2     fable1                110
                         fable2                390
gpt-4         fable1     fable1                 81
                         fable2                252
              fable2     fable1                306
                         fable2                332
              none       none                    3
              unknown    unknown                 5